In [1]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_importance_neg.csv', index_col=0)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
# gridsearch
for i in [128, 256, 512]:  # layer
    for p in ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']:  # optimizers
        tf.set_random_seed(42)

        # make keras model
        start = time.time()
        inputs = Input(shape=(X_train.shape[1],))
        x = Dense(i, activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(i, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(i, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(i, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)
        
        # compile
        model.compile(
            loss='categorical_crossentropy',
            optimizer=p,
            metrics=['accuracy']
        )

        epochs = 100
        batch_size = 1000
        es = EarlyStopping(monitor='val_loss', patience=20)

        history = model.fit(
            X_train,
            y_train_for_keras,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test_for_keras),
            verbose=0,
            callbacks=[
                es,
            ]
        )
        
        print(i, p, model.evaluate(X_test, y_test_for_keras, verbose=0)[1])
        
        if model.evaluate(X_test, y_test_for_keras, verbose=0)[1] > load_model('../model/Keras_fs_neg.h5').evaluate(X_test, y_test_for_keras, verbose=0)[1]:
            model.save('../model/Keras_fs_neg.h5')

128 SGD 0.5425531902211778
128 RMSprop 0.6170212727911929
128 Adagrad 0.5851063817105395
128 Adadelta 0.5638297948431461
128 Adam 0.5744680927154866
128 Adamax 0.5638297859658586
128 Nadam 0.6489361765536856
256 SGD 0.5425531902211778
256 RMSprop 0.6063829850643239
256 Adagrad 0.659574474426026
256 Adadelta 0.6276595719317173
256 Adam 0.6170212740593768
256 Adamax 0.595744678314696
256 Nadam 0.6170212740593768
512 SGD 0.5531914880935181
512 RMSprop 0.7234042616600685
512 Adagrad 0.6382978786813452
512 Adadelta 0.6276595820771887
512 Adam 0.6063829761870364
512 Adamax 0.6063829761870364
512 Nadam 0.6276595719317173


In [5]:
# tf.set_random_seed(42)

# # make keras model
# start = time.time()
# inputs = Input(shape=(X_train.shape[1],))

# x = Dense(512, activation='relu')(inputs)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# predictions = Dense(
#     len(df['Subclass'].value_counts()), activation='softmax')(x)
# model = Model(inputs=inputs, outputs=predictions)

# # compile
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adamax(),
#     metrics=['accuracy']
# )

# epochs = 100
# batch_size = 1000
# es = EarlyStopping(monitor='val_loss', patience=20)

# history = model.fit(
#     X_train,
#     y_train_for_keras,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test_for_keras),
#     verbose=1,
#     callbacks=[
#         es,
#     ]
# )

# model.save('../model/Keras_fs_neg.h5')